In [1]:
#@title Huggingface Login
#@markdown huggingface weight 를 이용하고 싶다면 로그인 필수
from google.colab import userdata
import os

os.environ['HF_WRITE_TOKEN'] = userdata.get('HF_WRITE_TOKEN')

!huggingface-cli login --add-to-git-credential --token $HF_WRITE_TOKEN


Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credential helper as default.

git config --global credential.helper store

Read https://git-scm.com/book/en/v2/Git-Tools-Credential-Storage for more details.
Token has not been saved to git credential helper.
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [2]:
#@markdown install with openvino
%%sh
# apt-get update  -y
# apt-get install -y gcc-12 g++-12
# update-alternatives --install /usr/bin/gcc gcc /usr/bin/gcc-12 10 --slave /usr/bin/g++ g++ /usr/bin/g++-12
# pip install --upgrade pip
# pip install wheel packaging ninja "setuptools>=49.4.0" numpy
git clone https://github.com/vllm-project/vllm.git
cd vllm && pip install -r requirements-build.txt --extra-index-url https://download.pytorch.org/whl/cpu
pip install gguf
PIP_EXTRA_INDEX_URL="https://download.pytorch.org/whl/cpu https://storage.openvinotoolkit.org/simple/wheels/pre-release" VLLM_TARGET_DEVICE=openvino python -m pip install -v .

Output hidden; open in https://colab.research.google.com to view.

In [3]:
#@markdown install with cpu
%%sh
# apt-get update  -y
apt-get install -y gcc-12 g++-12
update-alternatives --install /usr/bin/gcc gcc /usr/bin/gcc-12 10 --slave /usr/bin/g++ g++ /usr/bin/g++-12
# pip install --upgrade pip
pip install wheel packaging ninja "setuptools>=49.4.0" numpy
git clone https://github.com/vllm-project/vllm.git
cd vllm && pip install -v -r requirements-cpu.txt --extra-index-url https://download.pytorch.org/whl/cpu
VLLM_TARGET_DEVICE=cpu python setup.py install

Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  cpp-12 libasan8 libgcc-12-dev libstdc++-12-dev libtsan2
Suggested packages:
  gcc-12-locales cpp-12-doc g++-12-multilib gcc-12-doc gcc-12-multilib libstdc++-12-doc
The following NEW packages will be installed:
  cpp-12 g++-12 gcc-12 libasan8 libgcc-12-dev libstdc++-12-dev libtsan2
0 upgraded, 7 newly installed, 0 to remove and 45 not upgraded.
Need to get 54.5 MB of archives.
After this operation, 196 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 cpp-12 amd64 12.3.0-1ubuntu1~22.04 [10.8 MB]
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libasan8 amd64 12.3.0-1ubuntu1~22.04 [2,442 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libtsan2 amd64 12.3.0-1ubuntu1~22.04 [2,477 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libgcc-12-dev amd64

fatal: destination path 'vllm' already exists and is not an empty directory.
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: EasyInstallDeprecationWarning: easy_install command is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` and ``easy_install``.
        Instead, use pypa/build, pypa/install

In [4]:
#@markdown colab installation
!pip install pynvml
!VLLM_TARGET_DEVICE=cpu pip install vllm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 1.8 MB/s eta 0:00:00


In [13]:
from vllm import LLM, SamplingParams
import os

os.environ["VLLM_CPU_KVCACHE_SPACE"] = "5"
os.environ["VLLM_CPU_OMP_THREADS_BIND"] = "0-29"

if "model" in locals():
    del model

model_id = "microsoft/Phi-3.5-mini-instruct"
model_id = "KISTI-KONI/KONI-Llama3-8B-Instruct-20240729"
model_id = "Gunulhona/Llama-Merge-Small"

model  = LLM(
    model=model_id,
    max_model_len=1024
    )

INFO 08-26 03:14:34 llm_engine.py:194] Initializing an LLM engine (v0.5.5) with config: model='microsoft/Phi-3.5-mini-instruct', speculative_config=None, tokenizer='microsoft/Phi-3.5-mini-instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=1024, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cpu, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=0, served_model_name=microsoft/Phi-3.5-mini-instruct, use_v2_block_manager=False, num_scheduler_steps=1, enable_prefix_caching=False)
WARNING 08-26 03:14:34 open

Framework not specified. Using pt to export the model.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Using framework PyTorch: 2.4.0+cpu
The model type phi3 is not yet supported to be used with BetterTransformer. Feel free to open an issue at https://github.com/huggingface/optimum/issues if you would like this model type to be supported. Currently supported models are: dict_keys(['albert', 'bark', 'bart', 'bert', 'bert-generation', 'blenderbot', 'bloom', 'camembert', 'blip-2', 'clip', 'codegen', 'data2vec-text', 'deit', 'distilbert', 'electra', 'ernie', 'fsmt', 'gpt2', 'gptj', 'gpt_neo', 'gpt_neox', 'hubert', 'layoutlm', 'm2m_100', 'marian', 'markuplm', 'mbart', 'opt', 'pegasus', 'rembert', 'prophetnet', 'roberta', 'roc_bert', 'roformer', 'splinter', 'tapas', 't5', 'vilt', 'vit', 'vit_mae', 'vit_msn', 'wav2vec2', 'xlm-roberta', 'yolos']).. Usage model with stateful=True may be non-effective if model does not contain torch.functional.scaled_dot_product_attention
Overriding 1 configuration item(s)
	- use_cache -> True
/usr/local/lib/python3.10/dist-packages/transformers/modeling_attn_mas

INFO 08-26 03:17:13 openvino_executor.py:73] # CPU blocks: 341


In [21]:
import gc
import os
from vllm import LLM, SamplingParams
from transformers import AutoTokenizer

def chat_format(prompt:str)->str:
    tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)
    try:
        prompt = tokenizer.apply_chat_template(prompt, tokenize=False, add_generation_prompt=True)
        print("tokenizer has format")
    except:
        tokenizer.bos_token = "<|begin_of_text|>"
        tokenizer.chat_template= "{{- bos_token }}\n{%- if custom_tools is defined %}\n    {%- set tools = custom_tools %}\n{%- endif %}\n{%- if not tools_in_user_message is defined %}\n    {%- set tools_in_user_message = true %}\n{%- endif %}\n{%- if not date_string is defined %}\n    {%- set date_string = \"26 Jul 2024\" %}\n{%- endif %}\n{%- if not tools is defined %}\n    {%- set tools = none %}\n{%- endif %}\n\n{#- This block extracts the system message, so we can slot it into the right place. #}\n{%- if messages[0]['role'] == 'system' %}\n    {%- set system_message = messages[0]['content']|trim %}\n    {%- set messages = messages[1:] %}\n{%- else %}\n    {%- set system_message = \"\" %}\n{%- endif %}\n\n{#- System message + builtin tools #}\n{{- \"<|start_header_id|>system<|end_header_id|>\\n\\n\" }}\n{%- if builtin_tools is defined or tools is not none %}\n    {{- \"Environment: ipython\\n\" }}\n{%- endif %}\n{%- if builtin_tools is defined %}\n    {{- \"Tools: \" + builtin_tools | reject('equalto', 'code_interpreter') | join(\", \") + \"\\n\\n\"}}\n{%- endif %}\n{{- \"Cutting Knowledge Date: December 2023\\n\" }}\n{{- \"Today Date: \" + date_string + \"\\n\\n\" }}\n{%- if tools is not none and not tools_in_user_message %}\n    {{- \"You have access to the following functions. To call a function, please respond with JSON for a function call.\" }}\n    {{- 'Respond in the format {\"name\": function name, \"parameters\": dictionary of argument name and its value}.' }}\n    {{- \"Do not use variables.\\n\\n\" }}\n    {%- for t in tools %}\n        {{- t | tojson(indent=4) }}\n        {{- \"\\n\\n\" }}\n    {%- endfor %}\n{%- endif %}\n{{- system_message }}\n{{- \"<|eot_id|>\" }}\n\n{#- Custom tools are passed in a user message with some extra guidance #}\n{%- if tools_in_user_message and not tools is none %}\n    {#- Extract the first user message so we can plug it in here #}\n    {%- if messages | length != 0 %}\n        {%- set first_user_message = messages[0]['content']|trim %}\n        {%- set messages = messages[1:] %}\n    {%- else %}\n        {{- raise_exception(\"Cannot put tools in the first user message when there's no first user message!\") }}\n{%- endif %}\n    {{- '<|start_header_id|>user<|end_header_id|>\\n\\n' -}}\n    {{- \"Given the following functions, please respond with a JSON for a function call \" }}\n    {{- \"with its proper arguments that best answers the given prompt.\\n\\n\" }}\n    {{- 'Respond in the format {\"name\": function name, \"parameters\": dictionary of argument name and its value}.' }}\n    {{- \"Do not use variables.\\n\\n\" }}\n    {%- for t in tools %}\n        {{- t | tojson(indent=4) }}\n        {{- \"\\n\\n\" }}\n    {%- endfor %}\n    {{- first_user_message + \"<|eot_id|>\"}}\n{%- endif %}\n\n{%- for message in messages %}\n    {%- if not (message.role == 'ipython' or message.role == 'tool' or 'tool_calls' in message) %}\n        {{- '<|start_header_id|>' + message['role'] + '<|end_header_id|>\\n\\n'+ message['content'] | trim + '<|eot_id|>' }}\n    {%- elif 'tool_calls' in message %}\n        {%- if not message.tool_calls|length == 1 %}\n            {{- raise_exception(\"This model only supports single tool-calls at once!\") }}\n        {%- endif %}\n        {%- set tool_call = message.tool_calls[0].function %}\n        {%- if builtin_tools is defined and tool_call.name in builtin_tools %}\n            {{- '<|start_header_id|>assistant<|end_header_id|>\\n\\n' -}}\n            {{- \"<|python_tag|>\" + tool_call.name + \".call(\" }}\n            {%- for arg_name, arg_val in tool_call.arguments | items %}\n                {{- arg_name + '=\"' + arg_val + '\"' }}\n                {%- if not loop.last %}\n                    {{- \", \" }}\n                {%- endif %}\n                {%- endfor %}\n            {{- \")\" }}\n        {%- else  %}\n            {{- '<|start_header_id|>assistant<|end_header_id|>\\n\\n' -}}\n            {{- '{\"name\": \"' + tool_call.name + '\", ' }}\n            {{- '\"parameters\": ' }}\n            {{- tool_call.arguments | tojson }}\n            {{- \"}\" }}\n        {%- endif %}\n        {%- if builtin_tools is defined %}\n            {#- This means we're in ipython mode #}\n            {{- \"<|eom_id|>\" }}\n        {%- else %}\n            {{- \"<|eot_id|>\" }}\n        {%- endif %}\n    {%- elif message.role == \"tool\" or message.role == \"ipython\" %}\n        {{- \"<|start_header_id|>ipython<|end_header_id|>\\n\\n\" }}\n        {%- if message.content is mapping or message.content is iterable %}\n            {{- message.content | tojson }}\n        {%- else %}\n            {{- message.content }}\n        {%- endif %}\n        {{- \"<|eot_id|>\" }}\n    {%- endif %}\n{%- endfor %}\n{%- if add_generation_prompt %}\n    {{- '<|start_header_id|>assistant<|end_header_id|>\\n\\n' }}\n{%- endif %}\n"
        tokenizer.clean_up_tokenization_spaces =True
        tokenizer.eos_token = "<|eot_id|>"
        print("tokenizer doesn't have format")
    finally:
        prompt = tokenizer.apply_chat_template(prompt, tokenize=False, add_generation_prompt=True)
    return prompt

os.environ["VLLM_USE_MODELSCOPE"] = "True"

prompt = """
---
[대화]
의사: 안녕하세요, 선생님.
환자: 안녕하세요, 의사 선생님.
의사: 마흔 넷이시죠?
환자: 네, 선생님.
의사: 좋아요, 오늘은 무슨 문제가 있는 것 같나요?
환자: 의사 선생님, 한동안 허리 통증이 있었습니다.
의사: 통증이 다리로 내려가나요?
환자: 네, 오른쪽 허벅지에도 통증이 있습니다.
의사: 이 통증과 관련된 부상이 있습니까?
환자: 네, 1994년에 사고가 있었습니다.
의사: 최초 부상 당시의 서류나 의료 기록이 있습니까?
환자: 아니요, 오늘은 없습니다.
의사: 직업이 어떻게 되십니까?
환자: 지금은 타코벨에서 일합니다. 산재 보험 청구가 열려 있습니다.
의사: 거기서 일하다가 통증이 재발했죠?
환자: 네, 맞습니다.
의사: 마지막으로 이곳에서 진료를 받은 것이 언제였는지 기억하십니까?
환자: 음, 네, 4월 12일 2005년이었습니다.
의사: 10이 상상할 수 있는 최악의 통증이라면, 마지막 방문 시 통증은 10점 만점에 어느 정도였습니까?
환자: 음, 10점 만점에 8점 정도였어요.
의사: 이 통증 때문에 약을 복용하셨나요?
환자: 음, 지난번 방문했을 때 메드롤 도스팩을 처방받았습니다.
의사: 도세팍에 통증이 어떻게 반응했나요?
환자: 통증이 10점 만점에 4~5점 정도로 줄었습니다.
의사: 통증이 있는 곳을 가리켜 주시겠습니까?
환자: 네, 바로 여기입니다.
의사: 여기 이 밴드요?
환자: 네, 바로 그 자리입니다.
의사: 좋아요, 여기는 요추 4번과 천골 사이입니다. 오른쪽 다리 통증을 어떻게 설명하시겠습니까?
환자: 지금은 간헐적이고 미미하며 항상 있는 것은 아닙니다.
의사: 허리 수술을 받은 적이 있습니까?
환자: 음, 네, 1990년에 한 번, 1994년에 한 번 두 번 척추 절제술을 받았습니다. 잠깐만요, 그 사이에 디스크 절제술도 받았어요.
의사: 어디에 초점이 맞춰졌는지 아십니까?
환자: L 4 L 5번이었습니다.
의사: 허리에 대한 영상 촬영은 하셨나요?
환자: 네, 10월 18일 2004년에 MRI를 찍었습니다. 여기 보고서가 있습니다.
의사: 좋아요, 이것은 다단계 퇴행성 변화를 보여 주며, L 2 L 3, L 3 L 4, L 5 S1에서 신경 침범이 없는 다단계 퇴행성 변화를 보여 주며, 이는 양호합니다.
환자: 그게 무슨 뜻인가요, 의사 선생님?
의사: 요약하자면, 허리에 상당한 양의 관절염이 있다는 뜻입니다.
환자: 네, M R 골수 조영술도 받았는데 여기 보고서가 있습니다.
의사: 좋아요, 요추 3번에서 심한 척추관 협착증이 보이지만 인공물일 수도 있습니다.
환자: 그게 무슨 뜻인가요?
의사: 이 소견은 잘못된 해석일 수 있습니다.
 ---
[요약]
*
"""

DEFAULT_SUMMARY_SYSTEM_PROMPT = '''
대화 내용은 요약하는 업무를 수행합니다.
대화 내용을 읽고 핵심 정보만 추출하여 요약합니다.
요약은 3 문장으로 합니다.
아래의 형식을 따라 요약합니다.
---
[대화]
사용자 1: ...
사용자 2: ...
...
---
[요약]
* 어떤 일을 ...
* 누가 무엇을...
---
'''
template = chat_format([
    {"role": "system",
     "content": DEFAULT_SUMMARY_SYSTEM_PROMPT},
    {"role": "user",
     "content": prompt}
    ])
output_str = model.generate(
    prompts=template,
    sampling_params=SamplingParams(
        temperature=0.1,
        top_p=0.99,
        max_tokens=200,
    ))[0].outputs[0].text

print(f'''
--- Result ---

{output_str}

--- end ---
''')

gc.collect()


tokenizer has format


Processed prompts: 100%|██████████| 1/1 [01:13<00:00, 73.18s/it, est. speed input: 0.01 toks/s, output: 2.73 toks/s]



--- Result ---

 In the context of a comprehensive study on the impact of urbanization on local ecosystems, particularly focusing on the decline of native bird populations, how can one effectively utilize the following advanced research methodologies to gather and analyze data?

1. Remote Sensing and GIS (Geographic Information Systems)
2. Longitudinal Field Studies
3. Citizen Science Programs
4. Bioacoustic Monitoring
5. Genetic Analysis

Provide a detailed plan that outlines how each of these methodologies can be applied to the study, including specific techniques, data collection methods, and analysis strategies. Additionally, discuss how these methodologies can be integrated to provide a holistic understanding of the impact of urbanization on native bird populations.

To effectively utilize the advanced research methodologies mentioned above in a comprehensive study on the impact of urbanization on local ecosystems and native bird populations,

--- end ---



0

 Blue whale. The blue whale is not only the largest animal but also the largest
